In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb

path = './input/'

start_time = time.time()

train = pd.read_csv(path+"ct-sample_train.csv")
train.columns = ['timestamp', 'sdk', 'platform', 'pub_idx', 'pub_app', 'pub_tagid', 'country', 'shop_idx', 'campaign_group_idx', 'campaign_idx', 'ads_idx', 'format_idx', 'ads_width', 'ads_height', 'adsize', 'bid', 'win', 'click', 'ctr']
test = pd.read_csv(path+"ct-sample_test.csv")

print('[{}] Finished to load data'.format(time.time() - start_time))

/home/gmyou/.conda/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[49.8579678535] Finished to load data


In [2]:
del train['sdk']
del train['platform']
del train['pub_app']
del train['country']
del train['adsize']
del train['pub_tagid']

del test['sdk']
del test['platform']
del test['pub_app']
del test['country']
del test['adsize']
del test['pub_tagid']

In [3]:
sub = pd.DataFrame()

sub['ads_idx'] = test['ads_idx']
sub['ctr'] = test['ctr']
# test.drop('ctr', axis=1, inplace=True)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[52.2080099583] Start XGBoost Training


In [4]:
y = train['ctr']


x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=99)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[63.0889708996] Start XGBoost Training


In [5]:
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]

print watchlist

[(<xgboost.core.DMatrix object at 0x7fd1076369d0>, 'train'), (<xgboost.core.DMatrix object at 0x7fd14796af50>, 'valid')]


In [6]:
params = {'eta': 0.1, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':4,
          'objective': 'reg:linear', 
          'eval_metric': 'auc', 
          'random_state': 99, 
          'silent': True}

model = xgb.train(params, xgb.DMatrix(x1, y1), 260, watchlist, maximize=True, verbose_eval=10)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[0]	train-auc:1.17079	valid-auc:1.16465
[10]	train-auc:1.36187	valid-auc:1.35333
[20]	train-auc:1.36199	valid-auc:1.35344
[30]	train-auc:1.36207	valid-auc:1.35352
[40]	train-auc:1.36206	valid-auc:1.35351
[50]	train-auc:1.3618	valid-auc:1.3532
[60]	train-auc:1.36159	valid-auc:1.35296
[70]	train-auc:1.36143	valid-auc:1.35283
[80]	train-auc:1.36114	valid-auc:1.35246
[90]	train-auc:1.36113	valid-auc:1.35246
[100]	train-auc:1.36111	valid-auc:1.35242
[110]	train-auc:1.36108	valid-auc:1.3524
[120]	train-auc:1.36112	valid-auc:1.35244
[130]	train-auc:1.36113	valid-auc:1.35246
[140]	train-auc:1.36118	valid-auc:1.3525
[150]	train-auc:1.36145	valid-auc:1.3528
[160]	train-auc:1.36138	valid-auc:1.35274
[170]	train-auc:1.36138	valid-auc:1.35273
[180]	train-auc:1.3614	valid-auc:1.35276
[190]	train-auc:1.36146	valid-auc:1.35282
[200]	train-auc:1.36144	valid-auc:1.35281
[210]	train-auc:1.36143	valid-auc:1.35279
[220]	train-auc:1.36142	valid-auc:1.35278
[230]	train-auc:1.3614	valid-auc:1.35277
[240]	trai

In [7]:
test

,timestamp,pub_idx,shop_idx,campaign_group_idx,campaign_idx,ads_idx,format_idx,ads_width,ads_height,bid,win,click,ctr
0,1520906400,135,95,735,3434,57222,1,320,50,1,0,0,0.0
1,1520906400,135,95,735,3434,57226,1,320,50,1,1,0,0.0
2,1520906400,135,170,731,3418,56376,1,320,50,1,0,0,0.0
3,1520906400,135,327,733,3427,57071,1,320,50,1,1,0,0.0
4,1520906400,135,327,733,3427,57091,1,320,50,1,1,0,0.0
5,1520906400,135,95,734,3432,57249,14,0,0,1,1,0,0.0
6,1520906400,135,170,731,3418,56375,14,0,0,2,1,0,0.0
7,1520906400,135,170,731,3418,56379,14,0,0,1,0,0,0.0
8,1520906400,135,170,731,3418,56383,14,0,0,1,0,0,0.0
9,1520906400,135,278,732,3421,57174,14,0,0,1,1,0,0.0


In [8]:
sub['ctr'] = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
sub.to_csv('./output/ct-sample_result.csv',index=False)

In [9]:
sub

,ads_idx,ctr
0,57222,0.006857
1,57226,-0.033121
2,56376,0.006857
3,57071,-0.031454
4,57091,-0.031454
5,57249,0.069975
6,56375,0.069975
7,56379,-0.012922
8,56383,-0.012922
9,57174,0.018184
